CHROMA DB STORING CLASS

In [78]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
class CustomEmbeddingModel:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name,device = "cpu")

    @classmethod
    def save_vectors_to_chromadb(cls, embeddings, documents, db_path):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        try:
            client.delete_collection("testdb1")
        except:
            pass
        collection = client.create_collection("testdb1")
    
        collection.add(embeddings=embeddings, documents=documents, ids=[f"id{i}" for i in range(1, len(embeddings) + 1)])
        print(f"ChromaDB saved successfully to {db_path}")

    def encode_text(self, text):
        embeddings = self.model.encode(text, convert_to_tensor=True)
        return embeddings.tolist()

    def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks



STORING IN CHROMA DB

In [79]:
import asyncio
async def store_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        file_contents = file.read()
    
    segmented_text = CustomEmbeddingModel.segment_text(file_contents, chunk_size=50)
    # List of models
    model_names = [
        'sentence-transformers/all-mpnet-base-v2',
        "BAAI/bge-large-en-v1.5",
        'sentence-transformers/all-MiniLM-L6-v2',
        "thenlper/gte-base",
          "BAAI/bge-base-en-v1.5"  # Add additional model paths if needed
    ]
    
    # Initialize models and store in a list
    models = [CustomEmbeddingModel(model_name=name) for name in model_names]
    
    def generate_batch():
        for i in range(1,6):
            yield i
    
    def embed_text(segmented_text,no,model):
        embeddings = model.encode_text(segmented_text)
        database_path = f"./embedded_v2/{file_name.split('.')[0]}_{no}"
        model.save_vectors_to_chromadb(embeddings, segmented_text, database_path)
    # Encode sentences for each model
    
    
    count = generate_batch()
    task = [embed_text(segmented_text,next(count),model) for model in models]
    await asyncio.gather(*task)
    return "success"


In [80]:
await store_data("grandcholan.txt")


ChromaDB saved successfully to ./embedded_v2/grandcholan_1
ChromaDB saved successfully to ./embedded_v2/grandcholan_2
ChromaDB saved successfully to ./embedded_v2/grandcholan_3
ChromaDB saved successfully to ./embedded_v2/grandcholan_4
ChromaDB saved successfully to ./embedded_v2/grandcholan_5


TypeError: An asyncio.Future, a coroutine or an awaitable is required

In [ ]:
CHROMA DB RETRIEVING CLASS

In [1]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer

import requests
from chromadb.utils import embedding_functions
class CustomRetriever:
    def __init__(self, model_name):
        self.model = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)

    def encode_text(self, text):
        embeddings = self.model.encode(text, convert_to_tensor=True)
        return embeddings.tolist()

    @classmethod
    def load_chromadb(self,db_path,user_input,embedder1):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.get_collection("testdb1",embedding_function=embedder1)
        # return collection  
        results = collection.query(
        query_texts=[user_input],
        n_results=2,
        )
        return results


RETRIEVING 

In [2]:
        
model_names = [
    'sentence-transformers/all-mpnet-base-v2',
    "BAAI/bge-large-en-v1.5",
    'sentence-transformers/all-MiniLM-L6-v2',
    "thenlper/gte-base",
      "BAAI/bge-base-en-v1.5"  # Add additional model paths if needed
]
# Initialize models and store in a list
models = [CustomRetriever(model_name=name) for name in model_names]

def get_context(query,path):
    context = []
    count = 1
    for j in models:
        database_path1 = f"./embedded_v2/{path}_{count}"
        similar_texts = CustomRetriever.load_chromadb(database_path1,user_input,j.model)
        context.extend(similar_texts["documents"][0])
        count += 1
    
    # context1 = [' '.join(inner_list) for inner_list in list(set(context))]
    # final_context = " ".join(context1)

    final_context = list(set(context))
    return final_context




VARIATION FOR QUERY

In [3]:
import requests
import re

def generate_queries(original_query):
    payload = {
      "question": original_query,
      "company": "GrandCholan",
      "domain": "foods and services"
    }
    
    response = requests.post("http://46.43.144.145:7010/variation_rag",json = payload)
    
    variation_text = response.json()
    
    pattern = re.compile(r"^([^\?]+)(\?.*)?$")
    generated_queries=[]
   
    for line in variation_text.splitlines():
        match = pattern.search(line)
        if match:
            question = match.group(1)
            generated_queries.append(question.strip()[3:])
    
    return generated_queries


GETTING TEXT FOR FIVE VARIATION FROM FIVE MODELS

In [7]:
# while True:
file_name = "grandcholan.txt"
path = file_name.split(".")[0]
user_input = input("ENTER THE QUERY : ")
queries = generate_queries(user_input)
contexts = []
for query in queries:
    contexts.extend(get_context(query,path))

print(len(set(contexts)))
print(len(contexts))
final_context = " ".join(list(set(contexts)))
    # print("============================================")
    # print(final_context)
    # print("============================================")
    
    # payload = {
    #   "message_content": user_input,
    #   "content": final_context,
    #   "company_name": "grandcholan",
    #   "domain": "Foods and services"
    # }
    
    # response = requests.post("http://46.43.144.145:7010/rulebased_bot",json=payload)
    # print(queries)
    # print(response.json(),"\n\n-----------------------------------------------------")
    
    

ENTER THE QUERY :  kothu parotta


4
16


In [64]:
payload = {
  "message_content": user_input,
  "content": final_context,
  "company_name": "grandcholan",
  "domain": "Foods and services"
}

response = requests.post("http://46.43.144.145:7010/rulebased_bot",json=payload)

print(response.json())

To order food from Grand Cholan, you can visit their website or use the Uber Eats app to browse and select your desired items. After placing your order, track it until it arrives at your location.


In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

# Your text chunks and query
text_chunks = contexts


query = "How are telecom companies adapting to 5G technology?"

# # Initialize a Sentence Transformer model (you can use different models)
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# # Encode text chunks
# encoded_text_chunks = model.encode(text_chunks)

# # Encode the query
# encoded_query = model.encode([query])

# # Apply KMeans clustering based on similarity to the query
# num_clusters = 5  # Number of clusters
# kmeans = KMeans(n_clusters=num_clusters)
# kmeans.fit(encoded_text_chunks)

# # Calculate similarity of text chunks to the query
# similarities = model.transform(text_chunks, encoded_query)

# # Get cluster labels for each text chunk based on similarity
# cluster_labels = kmeans.predict(encoded_text_chunks)

# # Organize text chunks by cluster
# clusters = {}
# for i, label in enumerate(cluster_labels):
#     if label not in clusters:
#         clusters[label] = []
#     clusters[label].append((text_chunks[i], similarities[i][0]))
# print(clusters)
# print("#####################################################################################")
# # Print clusters sorted by similarity to the query
# for cluster_id, texts in clusters.items():
#     sorted_texts = sorted(texts, key=lambda x: x[1], reverse=True)
#     print(f"Cluster {cluster_id + 1}:")
#     for text, similarity in sorted_texts:
#         print(f"Similarity: {similarity:.4f} - Text: {text}")
#     print("------")




from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# # Your text chunks and query
# text_chunks = [
#     "Text chunk 1 ...",
#     "Text chunk 2 ...",
#     # Add your 25 text chunks here...
# ]

# query = "Your query text goes here..."

# Initialize a Sentence Transformer model (you can use different models)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode text chunks and query
encoded_text_chunks = model.encode(text_chunks)
encoded_query = model.encode([query])

# Concatenate encoded query with text chunks
encoded_text_chunks = encoded_text_chunks.tolist()
encoded_text_chunks.append(encoded_query[0])

# Calculate cosine similarity matrix
cos_sim_matrix = cosine_similarity(encoded_text_chunks)

# Apply KMeans clustering
num_clusters = 5  # Number of clusters
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(cos_sim_matrix)

# Get cluster labels for each text chunk
cluster_labels = kmeans.labels_
print(cluster_labels)
# Organize text chunks by cluster
# Organize text chunks by cluster
clusters = {i: [] for i in range(num_clusters)}
for i, label in enumerate(cluster_labels[:-1]):  # Exclude the last item (query)
    clusters[label].append(text_chunks[i])

# Print clusters
for cluster_id, texts in clusters.items():
    print(f"Cluster {cluster_id + 1}:")
    for text in texts:
        print(text)
    print("------")

[0 4 2 1 0 4 2 1 0 4 2 1 0 4 2 1 3]
Cluster 1:
His ideologies are far beyond our knowledge. Through his organization in the field of art and architecture; in religion and literature, we can witness the work of a powerful force newly liberated by the progressive imperialism of the time.   Raja Raja Cholan was the very first Tamil king to come up with the great idea of formulating an official record of all the chief events of his rule and his achievements.  Not to blow his own trumpet but to make his successors follow the path of wisdom.  A man of great conduct and a patron of arts.
 It’s no wonder that this great King of many facets has a heart of gold. His personality is much praised than his heroic deeds.  He loved his family very dearly and the affection he lavished on his family members can be read through many of the literary works.   A trace of feminism can be spotted in his deeds, like giving first respect to his elder sister Kunthavai and his grand-aunt Sembiyan Mahadevi.  His e

/home/vectone/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# Your text chunks and query
# Your text chunks and query
# text_chunks = [
#     "5G technology promises ultra-fast internet speeds and low latency.",
#     "Telecom companies are investing heavily in infrastructure for better coverage.",
#     "Fiber-optic cables play a crucial role in high-speed internet transmission.",
#     "The evolution from 4G to 5G has transformed the way we connect and communicate.",
#     "Mobile network operators are expanding their services to rural areas.",
#     "Telecommunications revolutionized global connectivity and communication.",
#     "Satellite communication enables wide-reaching network coverage.",
#     "Telecom spectrum auctions determine frequency allocations for carriers.",
#     "VoIP technology has revolutionized voice communication over the internet.",
#     "Telecom regulatory bodies ensure fair competition and consumer protection.",
#     "Edge computing is enhancing network performance for telecom providers.",
#     "IoT devices rely on robust telecom infrastructure for seamless connectivity.",
#     "The telecom industry faces challenges in cybersecurity and data privacy.",
#     "Virtual Network Operators lease network infrastructure from major carriers.",
#     "The convergence of telecom and IT has led to innovative services.",
#     "Telecom towers are essential for signal transmission and reception.",
#     "Network slicing in 5G enables customized services for different applications.",
#     "Telecom APIs facilitate the integration of services across platforms.",
#     "Telecom billing systems manage subscriber charges and usage.",
#     "5G technology is poised to revolutionize industries beyond telecommunications.",
#     "Telecom analytics play a pivotal role in network optimization and planning.",
#     "Telecom infrastructure resilience is critical during natural disasters.",
#     "Telecom companies are exploring blockchain for secure transactions.",
#     "Telecom network virtualization reduces infrastructure costs and improves flexibility.",
#     "Telecom innovation drives digital transformation across sectors."
# ]
text_chunks = contexts

query = "How are telecom companies adapting to 5G technology?"

from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity


# Initialize a Sentence Transformer model (you can use different models)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode text chunks and query
encoded_text_chunks = model.encode(text_chunks)
encoded_query = model.encode([query])[0]  # Extract the query embedding

# Calculate cosine similarity matrix
cos_sim_matrix = cosine_similarity(encoded_text_chunks, [encoded_query])

# Apply KMeans clustering
num_clusters = 5  # Number of clusters
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(encoded_text_chunks)

# Get cluster labels for each text chunk
cluster_labels = kmeans.labels_

# Calculate centroids of clusters
cluster_centers = kmeans.cluster_centers_

# Calculate similarity between query and cluster centroids
centroid_similarities = cosine_similarity([encoded_query], cluster_centers)

# Find the most similar cluster
most_similar_cluster = centroid_similarities.argmax()

# Retrieve texts in the most similar cluster
similar_texts = [text_chunks[i] for i, label in enumerate(cluster_labels) if label == most_similar_cluster]

# Print the most similar cluster's texts
print(f"Most similar cluster ({most_similar_cluster + 1}):")
for text in similar_texts:
    print(text)

Most similar cluster (4):
எடுத்துக்காட்டாக சுமத்திராவின் மன்னர்  சைலேந்திரர்  நாகப்பட்டினத்தில் ஒரு புத்தகத்தை  கட்டிக்  கொள்வதற்கு அனுமதித்து  உதவியதோடன்றி  அது  செவ்வனே கட்டிமுடிக்க படுவதற்கு தேவையான முழு ஒத்துழைப்பையும் கொடுத்தார்.    இராசராசன் கோவில் கட்டிடக்கலைக்கு முன்மாதிரியாக திகழ்ந்தார்.  இவர் காலத்தில் தஞ்சையில் கட்டப்பட்ட  பெருவுடையார் ஆலயம் இதற்கான தலைசிறந்த எடுத்துக்காட்டாகும்.   பல்கலை வல்லவரான இம்மன்னர் சிறந்த அறிஞராக இருந்ததோடு மட்டுமன்றி கலை,  இலக்கிய புலவர்களுக்கு புரவலராக இருந்தார்.   இராசராசன் சாதனைகளை நினைவூட்டும் விதமாக இவர் பல பட்டங்களை சூட்டிக்கொண்டார்.   அவற்றுள் குறிப்பிடத்தக்கவை மும்முடிச்சோழன்,     சோழேந்திர  சிம்மன்,  சிங்களாந்தகன்,  ஜெயம்கொண்டார்,   தெலிங்க குலகாலன்,   ரவிகுல மாணிக்கன்.  சிவபாதசேகரன் ஆகியவை.
 3 Turnberry Quay, Docklands, London E14 9RD UK
Mon – Sun: 12pm – 3pm & 5pm – 10pm
Call:
020 7515 6888
|
Email: info@grandcholan.com
Special
|
 Blog
|
Privacy Policy
|
Cookies Policy
|
GDPR PolicyGRAND CHOLAN
Menu
Basket
Todays Deals
Contact Us
Login
S

/home/vectone/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/vectone/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
